In [1]:
from deps.dobot_api import DobotApiDashboard, DobotApiMove
from deps import utils
from deps import cv_core
import numpy as np
import matplotlib.pyplot as plt
import cv2
from pynput import keyboard

#Connect to robot sockets. 
dash = DobotApiDashboard('192.168.1.6', 29999)
move = DobotApiMove('192.168.1.6', 30003)

keys = utils.Keyboard(dash)
cont = cv_core.Contours()




In [2]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'

In [7]:
keys.execute()

Position saved!
Position saved!
Position saved!
Position saved!
Special key pressed: Key.esc


In [8]:
print("Detecting Anchors")
while True:
    cap = cv2.VideoCapture(0)
    cap = cv_core.set_res(cap, (1024, 768))
    centers = cont.wait_for_anchors(cap, show = True)
    cap.release()
    if len(centers) == 4:
        break


xys = [(arr[0], arr[1]) for arr in keys.coords]
pix_coor = utils.assign_corners(centers)
robot_coor = utils.assign_corners(xys, reverse=True)


features_mm_to_pixels_dict = {}
for key, value in robot_coor.items():
    features_mm_to_pixels_dict[value] = pix_coor[key]


tf_mtx = cv_core.compute_tf_mtx(features_mm_to_pixels_dict)

Detecting Anchors


'0,{},JointMovJ(0.000000,0.000000,0.000000,0.000000);'

In [ ]:
cap = cv2.VideoCapture(0)
cap = cv_core.set_res(cap, (1024, 768))
#out = cv2.VideoWriter('out.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 25, (640,480))
cont = cv_core.Contours()

idx = 0
prev_point = (0,0,0)
while(True):
    ret, frame = cap.read()

    pt, sorted_contours = cont.find_contours(frame, 3)
    a,b,r = pt
    plot_img = frame.copy()
    cv2.circle(plot_img, (a, b), r, (0, 255, 0), 2)
    cv2.circle(plot_img, (a, b), 1, (0, 0, 255), 3)
    #with_contours = cv2.drawContours(plot_img, sorted_contours, -1,(0,255, 0),2)

    for c in sorted_contours:
        # compute the center of the contour
        M = cv2.moments(c)
        cX = int(M["m10"] / M["m00"])
        cY = int(M["m01"] / M["m00"])
        # draw the contour and center of the shape on the image
        #cv2.drawContours(plot_img, [c], -1, (0, 255, 0), 2)
        cv2.circle(plot_img, (cX, cY), 2, (0, 0, 255), -1)
        cv2.putText(plot_img, f"{cX},{cY}", (cX - 20, cY - 20),
        cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 0), 1)
    #out.write(with_contours)

    if prev_point is pt:
        idx += 1
    else:
        idx = 0
    prev_point = pt


    if idx >= 30:
        message = 'LOCKED'
        color = (0,255,0)
    else:
        message = 'SEARCHING'
        color = (0,0,255)

    cv2.rectangle(plot_img, (15,10), (200, 30), (0, 0, 0), -1)
    cv2.putText(plot_img, "TARGET:", (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 1)
    cv2.putText(plot_img, f"{message}", (100,25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)


    cv2.imshow('frame',plot_img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
#out.release()
cv2.destroyAllWindows()

centers_cuboids = cont.contour_centers(sorted_contours)

dash.ClearError()
dash.EnableRobot()

utils.default_pos(move)
for center in centers_cuboids:
    X, Y, _ = tf_mtx @ (center[0], center[1], 1)
    move.MovL(X, Y, -70,  0)
    move.MovL(X, Y, -90,  0)
    move.MovL(X, Y, -70,  0)
utils.default_pos(move)

#dash.DisableRobot()

In [4]:
utils.report_error(dash)


|CODE: 50|
__________________________________________________
Possible reasons from ALARM SERVO:
Invalid velocity or acceleration values.
__________________________________________________
Possible reasons from ALARM CONTROLLER:
Joint3 overspeed.


In [13]:
dash.ClearError()
dash.EnableRobot()

'0,{},EnableRobot();'